In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from adamw import AdamW
from nadam import Nadam
from uoptim import UOptimizer


from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from utils_exp import plot_graphs

import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

PATH = './fMNIST_data'

In [2]:
train_data = datasets.FashionMNIST(PATH, train=True, download=True, transform=transforms.ToTensor())

# calculate statistics for fashion MNIST
global_norm_mean = train_data.train_data.float().div(255).mean().item()
global_norm_std = train_data.train_data.float().div(255).std().item()

In [3]:
# util to create loaders
mnist_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((global_norm_mean,), (global_norm_std,)),
           ])

def mnist(batch_size=50, valid=0, shuffle=True, transform=mnist_transform, path=PATH):
    test_data = datasets.FashionMNIST(path, train=False, download=True, transform=transform)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    train_data = datasets.FashionMNIST(path, train=True, download=True, transform=transform)
    if valid > 0:
        num_train = len(train_data)
        indices = list(range(num_train))
        split = num_train-valid
        np.random.shuffle(indices)

        train_idx, valid_idx = indices[:split], indices[split:]
        train_sampler = SubsetRandomSampler(train_idx)
        valid_sampler = SubsetRandomSampler(valid_idx)

        train_loader = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
        valid_loader = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
    
        return train_loader, valid_loader, test_loader
    else:
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
        return train_loader, test_loader


def plot_mnist(images, shape):
    fig = plt.figure(figsize=shape[::-1], dpi=80)
    for j in range(1, len(images) + 1):
        ax = fig.add_subplot(shape[0], shape[1], j)
        ax.matshow(images[j - 1, 0, :, :], cmap = matplotlib.cm.binary)
        plt.xticks(np.array([]))
        plt.yticks(np.array([]))
    plt.show()

In [4]:
train_loader, valid_loader, test_loader = mnist(valid=10000)

In [5]:
class ConvLayer(nn.Module):
    def __init__(self, size, padding=1, pool_layer=nn.MaxPool2d(2, stride=2),
                 bn=False, dropout=False, activation_fn=nn.ReLU()):
        super(ConvLayer, self).__init__()
        layers = []
        layers.append(nn.Conv2d(size[0], size[1], size[2], padding=padding))
        if pool_layer is not None:
            layers.append(pool_layer)
        if bn:
            layers.append(nn.BatchNorm2d(size[1]))
        if dropout:
            layers.append(nn.Dropout2d())
        layers.append(activation_fn)
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

In [6]:
class FullyConnected(nn.Module):
    def __init__(self, sizes, dropout=False, activation_fn=nn.Tanh):
        super(FullyConnected, self).__init__()
        layers = []
        
        for i in range(len(sizes) - 2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            if dropout:
                layers.append(nn.Dropout())
            layers.append(activation_fn())
        else: # нам не нужен дропаут и фнкция активации в последнем слое
            layers.append(nn.Linear(sizes[-2], sizes[-1]))
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

In [7]:
class Net(nn.Module):
    def __init__(self, batchnorm=False, dropout=False, optim_type='UAdam', **optim_params):
        super(Net, self).__init__()
        
        self._conv1 = ConvLayer([1, 16, 3], bn=batchnorm)
        self._conv2 = ConvLayer([16, 32, 3], bn=batchnorm, activation_fn=nn.Sigmoid())
        
        self.fc = FullyConnected([32*7*7, 10], dropout=dropout)
        
        self._loss = None
        
        if optim_type == 'SGD':
            self.optim = optim.SGD(self.parameters(), **optim_params)
        elif optim_type == 'Adadelta':
            self.optim = optim.Adadelta(self.parameters(), **optim_params)
        elif optim_type == 'RMSProp':
            self.optim = optim.RMSprop(self.parameters(), **optim_params)
        elif optim_type == 'Adam':
            self.optim = optim.Adam(self.parameters(), **optim_params)
        elif optim_type == 'AdamW':
            self.optim = AdamW(self.parameters(), **optim_params)
        elif optim_type == 'Nadam':
            self.optim = Nadam(self.parameters(), **optim_params)
        elif optim_type == 'Adamax':
            self.optim = optim.Adamax(self.parameters(), **optim_params)
        elif optim_type == 'UOptimizer':
            self.optim = UOptimizer(params = self.parameters(), **optim_params)
    
    def conv(self, x):
        x = self._conv1(x)
        x = self._conv2(x)
        return x
        
    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 32*7*7)
        x = self.fc(x)
        return x
    
    def loss(self, output, target, **kwargs):
        self._loss = F.cross_entropy(output, target, **kwargs)
        self._correct = output.data.max(1, keepdim=True)[1]
        self._correct = self._correct.eq(target.data.view_as(self._correct)).to(torch.float).cpu().mean()
        return self._loss

In [8]:
models = {
          # testing correctness of SGD
          'SGD': Net(True, False, 'SGD', lr=1e-3).to(device), 
          'UOSGD': Net(True, False, 'UOptimizer', lr=1e-3).to(device), 
    
          'SGD_momentum':Net(True, False, 'SGD', momentum=0.9, lr=1e-3).to(device),
          'UOSGD_momentum':Net(True, False, 'UOptimizer', use_exp_avg_norm = True, beta1_dump=0, lr=1e-3).to(device),
    
          'SGD_momentum_n':Net(True, False, 'SGD', momentum=0.9, nesterov=True,  lr=1e-3).to(device),
          'UOSGD_momentum_n':Net(True, False, 'UOptimizer', use_exp_avg_norm = True, beta1_dump=0,
                                 exp_avg_norm_type='nesterov', lr=1e-3).to(device),
    
          # testing RMSProp      
          'RMSProp': Net(True, False, 'RMSProp', lr=1e-4).to(device), 
          'UORMSProp': Net(True, False, 'UOptimizer', use_exp_avg_sq_norm = True,  lr=1e-4).to(device), 
          
          # testing Adadelta
          'Adadelta':Net(True, False, 'Adadelta', lr=1).to(device),
          'UOAdadelta':Net(True, False, 'UOptimizer', use_exp_avg_sq_norm = True, use_adadelta_lr=True, lr=1).to(device),
            
          # testing adam-like algoritms. 
          'Adam': Net(True, False, 'Adam', lr=1e-4).to(device), 
          'UOAdam': Net(True, False, 'UOptimizer', use_exp_avg_norm=True, 
                        use_exp_avg_sq_norm = True, use_bias_correction= True, lr=1e-4).to(device), 
    
          'Amsgrad':Net(True, False, 'Adam', lr=1e-4, amsgrad=True).to(device),
          'UOAmsgrad':Net(True, False, 'UOptimizer', use_exp_avg_norm=True, use_exp_avg_sq_norm = True, 
                            use_bias_correction=True, exp_avg_sq_norm_type='max_past_sq', lr=1e-4).to(device), 
          
          # AdamW is not included in Pytorch, so I used fastai implementation from here:
          # https://github.com/anandsaha/fastai.part1.v2/commit/159e1712e60f299e11c42caab35c726f367bcd61
          'AdamW':Net(True, False, 'AdamW', lr=1e-4, weight_decay=0.00025).to(device),
          'UOAdamW':Net(True, False, 'UOptimizer', use_exp_avg_norm=True, use_exp_avg_sq_norm = True, 
                        use_bias_correction= True, decouple_wd=True, lr=1e-4, weight_decay=0.00025).to(device),
    
          # Please note that pytorch Nadam is the official Nadam implementation for Keras translated to PyTorch
          # I use the classical Nadam formulas, so the results could be differ
          'Nadam':Net(True, False, 'Nadam', lr=1e-4).to(device),
          'UONadam':Net(True, False, 'UOptimizer', use_exp_avg_norm=True, use_exp_avg_sq_norm = True, 
                            use_bias_correction= True,exp_avg_norm_type='nesterov',  lr=1e-4).to(device),
          
          'Adamax':Net(True, False, 'Adamax', lr=1e-4).to(device),
          'UOAdamax':Net(True, False, 'UOptimizer', use_exp_avg_norm=True, use_exp_avg_sq_norm = True, 
                        use_bias_correction= True, exp_avg_sq_norm_type ='infinite_l', lr=1e-4).to(device),
    
    
          # create exotic combinations for fun
          'Adam_with_adadelta_coeff':Net(True, False, 'UOptimizer', 
                                         use_exp_avg_norm = True,
                                         use_exp_avg_sq_norm = True,
                                         use_adadelta_lr = True,
                                         use_bias_correction = True,
                                         lr=1).to(device),
          'Adam_with_exp_avg_norm_like_sgd':Net(True, False, 'UOptimizer', 
                                         use_exp_avg_norm = True,
                                         use_exp_avg_sq_norm = True,
                                         use_bias_correction = True,
                                         beta1_dump=0,
                                         lr=1e-4).to(device),                  
            }
train_log = {k: [] for k in models}
test_log = {k: [] for k in models}

You are using the AdamW optimizer


In [9]:
def train(epoch, models, log=None):
    train_size = len(train_loader.sampler)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        for model in models.values():
            model.optim.zero_grad()
            output = model(data)
            loss = model.loss(output, target)
            loss.backward()
            model.optim.step()
            
        if batch_idx % 200 == 0:
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(train_loader))
            losses = ' '.join(['{}: {:.4f}'.format(k, m._loss.item()) for k, m in models.items()])
            print(line + losses)
            
    else:
        batch_idx += 1
        line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
            epoch, batch_idx * len(data), train_size, 100. * batch_idx / len(train_loader))
        losses = ' '.join(['{}: {:.4f}'.format(k, m._loss.item()) for k, m in models.items()])
        if log is not None:
            for k in models:
                log[k].append((models[k]._loss, models[k]._correct))
        print(line + losses)

In [10]:
def test(models, loader, log=None):
    test_size = len(loader.sampler)
    avg_lambda = lambda l: 'Loss: {:.4f}'.format(l)
    acc_lambda = lambda c, p: 'Accuracy: {}/{} ({:.0f}%)'.format(c, test_size, p)
    line = lambda i, l, c, p: '{}: '.format(i) + avg_lambda(l) + '\t' + acc_lambda(c, p)

    test_loss = {k: 0. for k in models}
    correct = {k: 0. for k in models}
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = {k: m(data) for k, m in models.items()}
            for k, m in models.items():
                test_loss[k] += m.loss(output[k], target, size_average=False).item() # sum up batch loss
                pred = output[k].data.max(1, keepdim=True)[1] # get the index of the max log-probability
                correct[k] += pred.eq(target.data.view_as(pred)).cpu().sum().item()
    
    for k in models:
        test_loss[k] /= test_size
    correct_pct = {k: c / test_size for k, c in correct.items()}
    lines = '\n'.join([line(k, test_loss[k], correct[k], 100*correct_pct[k]) for k in models]) + '\n'
    report = 'Test set:\n' + lines
    if log is not None:
        for k in models:
            log[k].append((test_loss[k], correct_pct[k]))
    print(report)

In [ ]:
n_epoch = 100
for epoch in range(1, n_epoch+1):
    for model in models.values():
        model.train()
    train(epoch, models, train_log)
    for model in models.values():
        model.eval()
    test(models, valid_loader, test_log)

Train Epoch: 1 [0/50000 (0%)]	Losses SGD: 2.2999 UOSGD: 2.3099 SGD_momentum: 2.3422 UOSGD_momentum: 2.4473 SGD_momentum_n: 2.3531 UOSGD_momentum_n: 2.3529 RMSProp: 2.3842 UORMSProp: 2.3295 Adadelta: 2.3630 UOAdadelta: 2.3002 Adam: 2.3547 UOAdam: 2.3272 Amsgrad: 2.3787 UOAmsgrad: 2.3775 AdamW: 2.2968 UOAdamW: 2.3983 Nadam: 2.3808 UONadam: 2.3416 Adamax: 2.3560 UOAdamax: 2.2726 Adam_with_adadelta_coeff: 2.3437 Adam_with_exp_avg_norm_like_sgd: 2.3443
Train Epoch: 1 [10000/50000 (20%)]	Losses SGD: 2.2181 UOSGD: 2.1376 SGD_momentum: 1.2718 UOSGD_momentum: 1.3821 SGD_momentum_n: 1.1687 UOSGD_momentum_n: 1.4108 RMSProp: 1.6016 UORMSProp: 0.9939 Adadelta: 0.4743 UOAdadelta: 0.4952 Adam: 1.7145 UOAdam: 1.7632 Amsgrad: 1.8000 UOAmsgrad: 1.7106 AdamW: 1.7846 UOAdamW: 1.7817 Nadam: 1.0715 UONadam: 1.7516 Adamax: 2.0142 UOAdamax: 2.0830 Adam_with_adadelta_coeff: 1.5639 Adam_with_exp_avg_norm_like_sgd: 0.5925
Train Epoch: 1 [20000/50000 (40%)]	Losses SGD: 2.1632 UOSGD: 2.0436 SGD_momentum: 0.9874 UO

Train Epoch: 3 [0/50000 (0%)]	Losses SGD: 1.6245 UOSGD: 1.3758 SGD_momentum: 0.5561 UOSGD_momentum: 0.5514 SGD_momentum_n: 0.5393 UOSGD_momentum_n: 0.5648 RMSProp: 0.6431 UORMSProp: 0.5524 Adadelta: 0.3040 UOAdadelta: 0.2961 Adam: 0.6634 UOAdam: 0.6451 Amsgrad: 0.6695 UOAmsgrad: 0.6376 AdamW: 0.8394 UOAdamW: 0.8492 Nadam: 0.6507 UONadam: 0.6328 Adamax: 0.9550 UOAdamax: 1.0228 Adam_with_adadelta_coeff: 0.4919 Adam_with_exp_avg_norm_like_sgd: 0.3953
Train Epoch: 3 [10000/50000 (20%)]	Losses SGD: 1.5079 UOSGD: 1.2643 SGD_momentum: 0.4701 UOSGD_momentum: 0.5037 SGD_momentum_n: 0.4751 UOSGD_momentum_n: 0.4982 RMSProp: 0.5583 UORMSProp: 0.4821 Adadelta: 0.3286 UOAdadelta: 0.2701 Adam: 0.5828 UOAdam: 0.5899 Amsgrad: 0.6057 UOAmsgrad: 0.5673 AdamW: 0.7567 UOAdamW: 0.7700 Nadam: 0.5548 UONadam: 0.5656 Adamax: 0.8287 UOAdamax: 0.8986 Adam_with_adadelta_coeff: 0.4396 Adam_with_exp_avg_norm_like_sgd: 0.2404
Train Epoch: 3 [20000/50000 (40%)]	Losses SGD: 1.4744 UOSGD: 1.2647 SGD_momentum: 0.5774 UO

## Visual results

#### Loss

In [ ]:
plot_graphs(test_log, 'loss', fs=(20, 10))

- Below we make a pairwise comparaison of standard algorithms and my implementation. Please note, that due to stochastic nature
of algorithms, the descent path could differ, but not too much. Exception for Nadam. For comparaison I use official implementation for Keras and it contains some hardcoded parameters that could not fit to any dataset. My implementation more classical and in case of this dataset looks better that Keras variant. 
- For better visualisation I excluded first 5 epochs, so below are the results from 6th epoch

In [ ]:
for i in ['SGD', 'SGD_momentum', 'SGD_momentum_n', 'RMSProp', 'Adadelta', 'Adam', 'Amsgrad', 'AdamW', 'Nadam', 'Adamax']:
    new_test_log = {}
    for k, v in test_log.items():
        if k == 'UO'+i or k==i:
            new_test_log[k] = v[1:]
    plot_graphs(new_test_log, 'loss')

In [ ]:
plot_graphs(test_log, 'accuracy', fs = (20, 10))

Now analyze dynamic of standard algorithms (we excluded the worst algos here like SGD, AdamW and Nadam). It looks like the best it is RMSProp here

In [ ]:
new_test_log = {}
for i in ['SGD_momentum', 'SGD_momentum_n', 'RMSProp', 'Adadelta', 'Adam', 'Amsgrad']:
    for k, v in test_log.items():
        if  k==i:
            new_test_log[k] = v[1:]
plot_graphs(new_test_log, 'loss', fs = (20, 10))

Lets compare RMSProp with the exotic combinations of our algorithm. **It looks like our exotic combinations provided better results that the best standard algoritm**

In [ ]:
new_test_log = {}
for i in ['RMSProp','Adam_with_adadelta_coeff', 'Adam_with_exp_avg_norm_like_sgd']:
    for k, v in test_log.items():
        if  k==i:
            new_test_log[k] = v[1:]
plot_graphs(new_test_log, 'loss', fs = (20, 10))